### Training of modified U-Net for Optic Disc on DRISHTI-GS database

In [ ]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import matplotlib.pyplot as plt
import h5py
import os

from model import *
from process_images import *
from process_result import *

print('Keras version:', keras.__version__)
print('TensorFlow version:', tf.__version__)

#### DRISHTI-GS

Carrega imagens de treino e teste com tamanho 512px.

In [ ]:
h5f = h5py.File(os.path.join(os.path.dirname(os.getcwd()), 'data', 'hdf5_datasets', 'DRISHTI_GS.hdf5'), 'a')
#-----------------------------------------------------------------------------------------------------------#
img_train = h5f['DRISHTI-GS/512 px/img_cropped_30']
cup_train = h5f['DRISHTI-GS/512 px/cup_cropped_30']
disc_train = h5f['DRISHTI-GS/512 px/disc_cropped_30']

img_test = h5f['DRISHTI-GS-test/512 px/img_cropped_30']
cup_test = h5f['DRISHTI-GS-test/512 px/cup_cropped_30']
disc_test = h5f['DRISHTI-GS-test/512 px/disc_cropped_30']

#### Generator of augmented data:

In [ ]:
gen = data_generator(img_train, cup_train, 128, 'train', batch_size=1)
batch = next(gen)
cur = batch[0][0].copy()
cur[np.where(batch[1][0])] /= 4
plt.imshow(cur)
plt.show()

In [ ]:
img_size = IMG_SIZE

X_test, Y_test = next(data_generator(img_test, cup_test, img_size,  train_or_test='test', batch_size=100, stationary=True))
len(X_test), len(Y_test)

In [ ]:
cur = X_test[27].copy()
cur[np.where(Y_test[27])] /= 4
plt.imshow(cur)
plt.show()

### Training and testing cup

In [ ]:
epochs = NUM_EPOCHS
spe = SPE
arq_name = 'lr_7_e_4_gap_70'
model = create_compile_model(img_size, 7e-4)
history_cup = train_cup(img_train, 
                    cup_train, 
                    'cup_color', 
                    model, epochs, X_test, Y_test, img_size, spe)

### Visual check

In [ ]:
pred_iou, pred_dice, pred_cup = predict(img_test, X_test, Y_test, model, img_size)

In [ ]:
folder_name = root_path(arq_name)

In [ ]:
print(np.mean(pred_iou))
print(np.mean(pred_dice))
save_iou_dice(np.mean(pred_iou), np.mean(pred_dice), folder_name, 'rgb_color_mean_iou_dice_cup.csv')

In [ ]:
X_test, Z_test = next(data_generator(img_test, disc_test, img_size, train_or_test='test', batch_size=100, stationary=True))

In [ ]:
cur = X_test[10].copy()
cur[np.where(Z_test[10])] /= 4
plt.imshow(cur)
plt.show()

In [ ]:
model_disc = create_compile_model(img_size, 1e-3)

history_disc = train_disc(img_train, 
                     disc_train, 
                     'disc_color', 
                     model_disc, epochs, X_test, Z_test, img_size, spe)

In [ ]:
pred_iou, pred_dice, pred_disc = predict(img_test, X_test, Z_test, model_disc, img_size)

In [ ]:
print(np.mean(pred_iou))
print(np.mean(pred_dice))
save_iou_dice(np.mean(pred_iou), np.mean(pred_dice), folder_name, 'rgb_color_mean_iou_dice_disc.csv')

In [ ]:
plot_results(history_cup, epochs, folder_name, arq_name+'_cup')

In [ ]:
plot_results(history_disc, epochs, folder_name, arq_name+'_disc')

In [ ]:
df, diametros = create_table_result(pred_cup, pred_disc, test_idx, folder_name, 'rgb_color')

In [ ]:
df

In [ ]:
diametros